In [1]:
%cd '..'

/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET


In [2]:
import inspect
import os
import re
import sys


import pandas as pd
from Bio import Entrez, SeqIO
from curated_set_services import CuratedSet
Entrez.email = "v.kulikov@intbio@org"

In [3]:
# Load data from histones.csv
curated_set = CuratedSet()
cs = curated_set

cs.data.shape, cs.data.columns

((565, 16),
 Index(['accession', 'type', 'variant_group', 'variant', 'doublet', 'gi',
        'ncbi_gene_id', 'hgnc_gene_name', 'taxonomy_id', 'organism', 'phylum',
        'class', 'taxonomy_group', 'info', 'references', 'sequence'],
       dtype='object'))

In [4]:
list(cs.has_duplicates())

[]

In [5]:
cs.data[cs.data["accession"] == "P02292.2"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence


In [6]:
cs.data.head()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_505463.1,NP_505463.1,H2A,cH2A,cH2A_(Animals),,17562014,,,6239,Caenorhabditis elegans,Nematoda,Chromadorea,,,26989147 22650316(?),MSGRGKGGKAKTGGKAKSRSSRAGLQFPVGRLHRILRKGNYAQRVG...
EEC09557.1,EEC09557.1,H2A,cH2A,cH2A_(Animals),,215500063,,,6945,Ixodes scapularis,Arthropoda,Arachnida,,,26989147 22650316(?),MSGRGKGGKVKGKSKTRSSRAGLQFPVGRIHRLLRKGNYAERVGAG...
NP_724343.1,NP_724343.1,H2A,cH2A,cH2A_(Animals),,24585673,,,7227,Drosophila melanogaster,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKVKGKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
XP_001119899.1,XP_001119899.1,H2A,cH2A,cH2A_(Animals),,110764935,,,7460,Apis mellifera,Arthropoda,Insecta,,,26989147 22650316(?),MSGRGKGGKAKAKAKSRSNRAGLQFPVGRIHRLLRKGNYAERVGAG...
EDO48405.1,EDO48405.1,H2A,cH2A,cH2A_(Animals),,156227602,,,45351,Nematostella vectensis,Cnidaria,Anthozoa,,,26989147 22650316(?),MSGRGKGKAKGTKSKTRSSRAGLQFPVGRIHRHLRKGNYAERVGAG...


In [7]:
cs.data.shape

(565, 16)

In [8]:
sperm_h2b_accessions = [
    "P02292.2",
]

In [9]:
df = pd.DataFrame(
    {
        "accession": sperm_h2b_accessions,
        "type": ["H2B"] * len(sperm_h2b_accessions),
        "variant_group": ["sperm_H2B_(Echinoidea)"] * len(sperm_h2b_accessions),
        "variant": ["sperm_H2B_(Echinoidea)"] * len(sperm_h2b_accessions),
        "references": ["710402"] * len(sperm_h2b_accessions),
    }
)
df.index = df.accession
df.shape, df.columns

((1, 5),
 Index(['accession', 'type', 'variant_group', 'variant', 'references'], dtype='object'))

In [10]:
df.head()

,accession,type,variant_group,variant,references
accession,,,,,
P02292.2,P02292.2,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),710402


In [11]:
cs.data = pd.concat([cs.data, df]).fillna("")
cs.data.shape, list(cs.has_duplicates())

((566, 16), [])

In [12]:
cs.data.tail()

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
NP_005314.2,NP_005314.2,H1,H1.6,H1.6_(Homo_sapiens)__???,,,3010.0,H1-6,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETVPAASASAGVAAMEKLPTKKRGRKPAGLISASRKVPNLSVSK...
NP_005313.1,NP_005313.1,H1,H1.5,H1.5_(Homo_sapiens)__???,,,3009.0,H1-5,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETAPAETATPAPVEKSPAKKKATKKAAGAGAAKRKATGPPVSEL...
NP_005316.1,NP_005316.1,H1,H1.1,H1.1_(Homo_sapiens)__???,,,3024.0,H1-1,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MSETVPPAPAASAAPEKPLAGKKAKKPAKAAAASKKKPAGPSVSEL...
NP_861453.1,NP_861453.1,H1,H1.7,H1.7_(Homo_sapiens)__???,,,341567.0,H1-7,9606,Homo sapiens,Chordata,Mammalia,Mammalia,,26689747,MEQALTGEAQSRWPRRGGSGAMAEAPGPSGESRGHSATQLPAEKTV...
P02292.2,P02292.2,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,,,,,,,,,,710402,


In [13]:
cs.data[cs.data["variant"] == "sperm_H2B_(Echinoidea)"].shape

(6, 16)

In [14]:
cs.update_accession_version()
cs.data = cs.data.set_index(cs.data.accession.values)

/home/l_singh/.conda/envs/histdb_env/lib/python3.8/site-packages/Bio/GenBank/__init__.py:1143: BiopythonParserWarning: Dropping bond qualifier in feature location
  warnings.warn(
/mnt/scratch/v_kulikov/histonedb_Vladimir/CURATED_SET/curated_set_services.py:267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updating_data['accession'] = new_accessions


In [15]:
cs.data[cs.data["variant"] == "sperm_H2B_(Echinoidea)"].shape

(6, 16)

In [16]:
curated_set.update_taxids(blank_data=True)

Fetched taxid from NCBI 7658
 changes to 7658
 changes to Parechinus angulosus
 changes to Echinodermata
 changes to Echinoidea


In [17]:
cs.data[cs.data["variant"] == "sperm_H2B_(Echinoidea)"].shape

(6, 16)

In [19]:
cs.data[cs.data["variant"] == "sperm_H2B_(Echinoidea)"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P02292.2,P02292.2,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,,,,7658,Parechinus angulosus,Echinodermata,Echinoidea,,,710402,
P02291.2,P02291.2,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,108885304,,,7658,Parechinus angulosus,Echinodermata,Echinoidea,,,,MPRSPAKTSPRKGSPRKGSPSRKASPKRGGKGAKRAGKGGRRRRVV...
Q27749.3,Q27749.3,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,74767039,,,7660,Psammechinus miliaris,Echinodermata,Echinoidea,,,,MPSQKSPTKRSPTKRSPQKGGKGAKRGGKAGKRRRGVAVKRRRRRR...
Q27750.3,Q27750.3,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,108860775,,,7660,Psammechinus miliaris,Echinodermata,Echinoidea,,,,MPKSPSKSSPRKGSPRKGSPRKGSPKRGGKGAKRAGKGGRRNVVKR...
NP_999706.1,NP_999706.1,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,47551059,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,,MPSQRSPTKRSPTKRSPQKGAGKGGKGSKRGGKARRRGGAAVRRRR...
NP_999721.1,NP_999721.1,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,47551083,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,,MPRSPSKTSPRKGSPRRGSPSRKASPKRGGKGAKRAGKGGRRRNVV...


## Updating sequences for sperm_H2B

In [20]:
cs.data[cs.data['sequence'] == '']

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P02292.2,P02292.2,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,,,,7658,Parechinus angulosus,Echinodermata,Echinoidea,,,710402,


In [21]:
cs.update_sequence(blank_data=True)

Fetching 1 seqs
Sequence for P02292.2 changes from  to MPRSPAKTSPRKGSPRKGSPRKGSPSRKASPKRGGKGAKRAGKGGRRRRVVKRRRRRRESYGIYIYKVLKQVHPDTGISSRAMSVMNSFVNDVFERIASEASRLTSANRRSTVSSREIQTAVRLLLPGELAKHAVSEGTKAVTKYTTSR
Sequences updated: 1


In [22]:
cs.data[cs.data["variant"] == "sperm_H2B_(Echinoidea)"]

,accession,type,variant_group,variant,doublet,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,references,sequence
P02292.2,P02292.2,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,,,,7658,Parechinus angulosus,Echinodermata,Echinoidea,,,710402,MPRSPAKTSPRKGSPRKGSPRKGSPSRKASPKRGGKGAKRAGKGGR...
P02291.2,P02291.2,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,108885304,,,7658,Parechinus angulosus,Echinodermata,Echinoidea,,,,MPRSPAKTSPRKGSPRKGSPSRKASPKRGGKGAKRAGKGGRRRRVV...
Q27749.3,Q27749.3,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,74767039,,,7660,Psammechinus miliaris,Echinodermata,Echinoidea,,,,MPSQKSPTKRSPTKRSPQKGGKGAKRGGKAGKRRRGVAVKRRRRRR...
Q27750.3,Q27750.3,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,108860775,,,7660,Psammechinus miliaris,Echinodermata,Echinoidea,,,,MPKSPSKSSPRKGSPRKGSPRKGSPKRGGKGAKRAGKGGRRNVVKR...
NP_999706.1,NP_999706.1,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,47551059,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,,MPSQRSPTKRSPTKRSPQKGAGKGGKGSKRGGKARRRGGAAVRRRR...
NP_999721.1,NP_999721.1,H2B,sperm_H2B_(Echinoidea),sperm_H2B_(Echinoidea),,47551083,,,7668,Strongylocentrotus purpuratus,Echinodermata,Echinoidea,,,,MPRSPSKTSPRKGSPRRGSPSRKASPKRGGKGAKRAGKGGRRRNVV...


In [23]:
cs.save()

Added sequence with accession P02292.2
cp histones.csv backups/histones.csv-Mar2624091216
Previous data backuped to backups/histones.csv-Mar2624091216
Results saved to histones.csv
